# Notebook for Toronto City clustering project

##### We will work on extracting borough & neighbourhood information + cleaning + clustering & visualising with Folium

#### 1. Establish environment

In [1]:
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re

#### 2. Ignore SSL errors

In [2]:
cntxt = ssl.create_default_context()
cntxt.check_hostname = False
cntxt.verify_mode = ssl.CERT_NONE

#### 3. Obtain URL

In [3]:
url = input('Please enter the website to obtain data from: ')
if len(url) < 1: url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Please enter the website to obtain data from:  


In [4]:
print('You want data from >>>\n', url, '\n<<<')

You want data from >>>
 https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 
<<<


#### 4. Open and parse the url

In [5]:
#Use a file-handle like object to open the url
html = urlopen(url, context= cntxt).read() #Read slurps everything in #Note that is additional function written at end

#Use BeautifulSoup to parse
soup = BeautifulSoup(html, 'html.parser')

#### 5. Explore

In [6]:
type(soup)

bs4.BeautifulSoup

In [7]:
#Try retrieving 'tr' tags
#print(soup)
tags = soup('tr')
print('Total tags extracted: ', len(tags),'\n')

#Look at the tag extracted
count = 0 #Initialise counter to count iterations & if rqd, help break out of loop
dict = {} #Initialise empty dictionary to store postcode as KEY, boroughs and neighbourhoods as VALUES
list = [] #Initialise empty list to hold the borough + neighbourhood info

#Loop through the tags
for i in tags:
    count = count + 1 #Increase counter value beginning through each iteration
    if count == 290: break #For limiting output and stopping the loop from running + length of tags for some reason does not correspond to actual number of elements
    
    #Skip if Borough is Not Assigned
    if i.contents[3].text == 'Not assigned':
        continue
        
    #Skip if text is Postcode
    if i.contents[1].text == 'Postcode':
        continue
    
    #If pincode already exists, eg M5A, M6A, append new data
    if i.contents[1].text in dict: 
        
        # Append the new data to the existing array at this slot with the following NEW SYNTAX
        #dict[existing_key].append(value)
        dict[i.contents[1].text].append(i.contents[5].text.rstrip())
        
    #If borough exists but not the neighbourhood, run the following    
    elif re.search('[a-z]', i.contents[3].text) and re.search('No.*', i.contents[5].text):
        list.append(i.contents[3].text), list.append(i.contents[3].text)
        dict[i.contents[1].text] = list
    else:
        list.append(i.contents[3].text)
        list.append(i.contents[5].text.rstrip())#rstrip gets rid of newline char.
        dict[i.contents[1].text] = list
    
    #Would need to reset the list to empty after each round otherwise each successive iteration will bloat up the key-value
    list = []
    

#print(dict)

print('\nTotal post-codes with borough info:', len(dict))

Total tags extracted:  294 


Total post-codes with borough info: 103


#### 6. Convert to pandas dataframe

In [8]:
#6.1 Import the library
import pandas as pd

In [9]:
#6.2 Convert dictionary to dataframe
data = pd.DataFrame.from_dict(dict, orient= 'index')#index when keys are row labels

In [10]:
data.head()

,0,1,2,3,4,5,6,7,8
M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,None,None,None,None,None
M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
M5B,Downtown Toronto,Ryerson,Garden District,None,None,None,None,None,None
M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,None,None,None
M3L,North York,Downsview West,None,None,None,None,None,None,None


In [11]:
#6.3 Assign row name to postcodes
data.index.name = 'PostalCode'
data.head()

,0,1,2,3,4,5,6,7,8
PostalCode,,,,,,,,,
M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,None,None,None,None,None
M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
M5B,Downtown Toronto,Ryerson,Garden District,None,None,None,None,None,None
M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,None,None,None
M3L,North York,Downsview West,None,None,None,None,None,None,None


In [12]:
#6.4 Reset index
data.reset_index(inplace= True)
data.head()

,PostalCode,0,1,2,3,4,5,6,7,8
0,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,None,None,None,None,None
1,M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
2,M5B,Downtown Toronto,Ryerson,Garden District,None,None,None,None,None,None
3,M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,None,None,None
4,M3L,North York,Downsview West,None,None,None,None,None,None,None


In [13]:
#6.5 Set column 2 i.e. after Postcode to Borough
data.columns.values[1] = 'Borough'#We need to drill down to the array, accessed with .values
data.head()

,PostalCode,Borough,1,2,3,4,5,6,7,8
0,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,None,None,None,None,None
1,M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
2,M5B,Downtown Toronto,Ryerson,Garden District,None,None,None,None,None,None
3,M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,None,None,None
4,M3L,North York,Downsview West,None,None,None,None,None,None,None


In [14]:
#6.6 Convert columns to list form so as to remove hidden characters that interfer with column extraction by name 
data.columns = data.columns.tolist()
data[['Borough']].head() #data[[5]] for the column numbered 5

,Borough
0,Scarborough
1,Etobicoke
2,Downtown Toronto
3,Etobicoke
4,North York


In [15]:
#6.7 Remove None values
## It is vital to try this in the beginning when only specific cells have None
### It becomes a bit complex if we try to remove None after merging, as each cell then has several values 

#data_sorted = data.apply(sorted,key=pd.isnull)
#data_sorted.head()
data_clean = data[~pd.isnull(data).all(1)].fillna('')
data_clean.head()

,PostalCode,Borough,1,2,3,4,5,6,7,8
0,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,,,,,
1,M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
2,M5B,Downtown Toronto,Ryerson,Garden District,,,,,,
3,M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,,,
4,M3L,North York,Downsview West,,,,,,,


In [16]:
#6.8 Concatenate the neighbourhood columns
## Obtain column position after which concatenation is to take place
source_col_loc = data_clean.columns.get_loc('Borough') # column position starts from 0
source_col_loc

#Create new column that will merge all neighbourhoods
data_clean['Neighbourhood'] = data_clean.iloc[:,source_col_loc+1:source_col_loc+8].apply(
   lambda x: ",".join(x.astype(str)), axis=1)
data_clean.head()

,PostalCode,Borough,1,2,3,4,5,6,7,8,Neighbourhood
0,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,,,,,,"Highland Creek,Rouge Hill,Port Union,,,,"
1,M9V,Etobicoke,Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
2,M5B,Downtown Toronto,Ryerson,Garden District,,,,,,,"Ryerson,Garden District,,,,,"
3,M9B,Etobicoke,Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park,,,,"Cloverdale,Islington,Martin Grove,Princess Gar..."
4,M3L,North York,Downsview West,,,,,,,,"Downsview West,,,,,,"


In [17]:
#6.9 Drop columns 1 through 8
import numpy as np
cols_to_remove = np.arange(1,9)
#print(cols_to_remove, '\n')

#Drop
data_clean.drop(cols_to_remove, axis=1, inplace = True)
#data_clean.head()

In [18]:
data_clean.head()

,PostalCode,Borough,Neighbourhood
0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,,,,"
1,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
2,M5B,Downtown Toronto,"Ryerson,Garden District,,,,,"
3,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
4,M3L,North York,"Downsview West,,,,,,"


In [19]:
data_clean[data_clean['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighbourhood
8,M5A,Downtown Toronto,"Harbourfront,Regent Park,,,,,"


In [20]:
data_clean.shape

(103, 3)

#### 7. Obtain geocoordinates of the postal codes

In [21]:
#import geocoder #Not installed
#From terminal, install GeoPy
#sudo python3 -m pip install geopy
#sudo python3 -m pip install geocoder #NOTE: THIS IS ANOTHER PACKAGE!

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium #Plotting library
#import geocoder

In [22]:
#g = geocoder.google('Mountain View, CA')
#print(g.latlng)

In [23]:
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
 #   g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  #  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [24]:
#Read direct from provided CSV as the package was not giving results
import pandas as pd
latlang = pd.read_csv('/Users/peaceful_warrior/Downloads/Geospatial_Coordinates.csv')

In [25]:
latlang.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
latlang.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [27]:
latlang.columns.values[0] = 'PostalCode'

In [28]:
latlang.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
data_clean.head()

,PostalCode,Borough,Neighbourhood
0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,,,,"
1,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
2,M5B,Downtown Toronto,"Ryerson,Garden District,,,,,"
3,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
4,M3L,North York,"Downsview West,,,,,,"


In [30]:
data_clean_sorted = data_clean.sort_values(by = 'PostalCode', ascending= True)
data_clean_sorted.head()

,PostalCode,Borough,Neighbourhood
88,M1B,Scarborough,"Rouge,Malvern,,,,,"
0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,,,,"
67,M1E,Scarborough,"Guildwood,Morningside,West Hill,,,,"
7,M1G,Scarborough,"Woburn,,,,,,"
40,M1H,Scarborough,"Cedarbrae,,,,,,"


In [31]:
data_clean_sorted.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [32]:
data_clean_sorted[['PostalCode']].head()

,PostalCode
88,M1B
0,M1C
67,M1E
7,M1G
40,M1H


In [33]:
#latlang[['PostalCode']].head()#Throws error, possibly due to hidden character, so convert to list

In [34]:
latlang.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [35]:
#Convert to list
latlang.columns = latlang.columns.tolist()

In [36]:
latlang[['PostalCode']].head()#Now it displays properly

,PostalCode
0,M1B
1,M1C
2,M1E
3,M1G
4,M1H


In [37]:
#Merge the two dataframes on PostalCode
data_merged = pd.merge(data_clean_sorted, latlang, on='PostalCode')

In [38]:
data_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern,,,,,",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,,,,",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,,,,",43.763573,-79.188711
3,M1G,Scarborough,"Woburn,,,,,,",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,,,,,,",43.773136,-79.239476


In [39]:
data_merged[data_merged['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
53,M5A,Downtown Toronto,"Harbourfront,Regent Park,,,,,",43.65426,-79.360636


In [40]:
#Check how many boroughs and neighbourhoods exist
print('There exists {} boroughs and {} neighbourhoods in Toronto'.format(
    len(data_merged['Borough'].unique()), 
      data_merged.shape[0]))

There exists 11 boroughs and 103 neighbourhoods in Toronto


#### 8. Use geopy library to get the latitude and longitude values of Toronto, Canada

In [41]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


#### 9. Create map of Toronto using latitude and longitude values

In [42]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode , borough, neighborhood in zip(data_merged['Latitude'], data_merged['Longitude'],
                                                      data_merged['PostalCode'],data_merged['Borough'],data_merged['Neighbourhood']):
    label = '{}, {}'.format(postcode, borough)#neighborhood, borough originally
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 10. Obtain info about the neighbourhoods

In [47]:
#10.2 Explore first postal code in df
data_merged.iloc[0:1,0:]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern,,,,,",43.806686,-79.194353


In [48]:
#10.3 Extract post code latitude & longitude
post_lat = data_merged.iloc[0,3]
post_long = data_merged.iloc[0,4]
print ('The coordinates of {} are lat:{} and long:{}'.format(data_merged.iloc[0,0], post_lat, post_long))

The coordinates of M1B are lat:43.806686299999996 and long:-79.19435340000001


In [49]:
#10.3 Extract top 20 venues in the post code of M1B within a radius of 500m 
limit = 20
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.\
format(CLIENT_ID, CLIENT_SECRET, post_lat,post_long, VERSION, radius, limit)
#url


In [50]:
#10.4 Send GET request and examine results
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1f3eb0018cbb00336204c3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [51]:
#10.5 Borrowing functions from previous labs to clean up the results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
 
#Convert JSON to dataframe
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [52]:
print('There are {} venues found in a radius of {}m around {}'.format(nearby_venues.shape[0], radius, data_merged.iloc[0,0]))

There are 2 venues found in a radius of 500m around M1B


*Let us keep radius as 500m for now, it makes sense for M1B that appears to be on the outskirts on the north-east**

In [53]:
#10.6 Borrow function to run above on all postcodes i.e. obtain venue category for all the postcodes

def getNearbyVenues(names, borough ,latitudes, longitudes, radius=500):#Where names = postcode
    
    venues_list=[]#Initialise empty list to store details
    for name, borough, lat, lng in zip(names, borough, latitudes, longitudes):
        print(name,'\t',borough)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)#LIMIT
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, #postcode
            borough,#I think this is useful to have, several post codes are in the same borough that may have similar char.
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    #Create new dataframe and fill it with values from the venue_list array with 2 nested for loops
                                                ##Note: placement of 'item' below
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode',#'Neighborhood' 
                             'Borough',
                  'Postcode Latitude',#'Neighborhood Latitude' 
                  'Postcode Longitude',#'Neighborhood Longitude' 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
#10.7 Run the above code

toronto_venues = getNearbyVenues(names=data_merged['PostalCode'],#['Neighborhood']
                                 borough = data_merged['Borough'],
                                   latitudes=data_merged['Latitude'],
                                   longitudes=data_merged['Longitude']
                                  )


M1B 	 Scarborough
M1C 	 Scarborough
M1E 	 Scarborough
M1G 	 Scarborough
M1H 	 Scarborough
M1J 	 Scarborough
M1K 	 Scarborough
M1L 	 Scarborough
M1M 	 Scarborough
M1N 	 Scarborough
M1P 	 Scarborough
M1R 	 Scarborough
M1S 	 Scarborough
M1T 	 Scarborough
M1V 	 Scarborough
M1W 	 Scarborough
M1X 	 Scarborough
M2H 	 North York
M2J 	 North York
M2K 	 North York
M2L 	 North York
M2M 	 North York
M2N 	 North York
M2P 	 North York
M2R 	 North York
M3A 	 North York
M3B 	 North York
M3C 	 North York
M3H 	 North York
M3J 	 North York
M3K 	 North York
M3L 	 North York
M3M 	 North York
M3N 	 North York
M4A 	 North York
M4B 	 East York
M4C 	 East York
M4E 	 East Toronto
M4G 	 East York
M4H 	 East York
M4J 	 East York
M4K 	 East Toronto
M4L 	 East Toronto
M4M 	 East Toronto
M4N 	 Central Toronto
M4P 	 Central Toronto
M4R 	 Central Toronto
M4S 	 Central Toronto
M4T 	 Central Toronto
M4V 	 Central Toronto
M4W 	 Downtown Toronto
M4X 	 Downtown Toronto
M4Y 	 Downtown Toronto
M5A 	 Downtown Toronto
M5B 	 Do

In [55]:
#10.8 Check size of resulting df
toronto_venues.shape

(1075, 8)

In [56]:
#10.9 Do a head
toronto_venues.head()

,Postcode,Borough,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1B,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,M1C,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,M1E,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [57]:
#10.10 Check how many venues were returned for each postcode (limit = 20)
toronto_venues.groupby('Postcode').count()

,Borough,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,,
M1B,2,2,2,2,2,2,2
M1C,2,2,2,2,2,2,2
M1E,9,9,9,9,9,9,9
M1G,4,4,4,4,4,4,4
M1H,8,8,8,8,8,8,8
M1J,1,1,1,1,1,1,1
M1K,4,4,4,4,4,4,4
M1L,10,10,10,10,10,10,10
M1M,3,3,3,3,3,3,3


In [58]:
#10.11 Find number of unique categories of venues
len(toronto_venues['Venue Category'].unique())

219

#### 11. Clustering

In [59]:
#11.1 Perform one-hot encoding as clustering algos cannot work on categorical variables directly
pd.set_option('display.max_columns', 300)

#One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()#Df of 1hot encoded venues, without info about postcode, lat,long etc
print(toronto_onehot.shape)

#Add postcode & borough info back to dataframe
#toronto_onehot['Postcode'] = toronto_venues['Postcode'] 
#toronto_onehot['Borough'] = toronto_venues['Borough']
#df.insert(2, 'new-col', data)
toronto_onehot.insert(0,'Postcode', toronto_venues['Postcode'])
toronto_onehot.head()
toronto_onehot.insert(1, 'Borough', toronto_venues['Borough'])
toronto_onehot.head()


(1075, 219)


,Postcode,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1B,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [60]:
#11.2 Obtain mean occurrence of categories of location at each place i.e. if there are 10 places and 5 of them are a restaurant, then mean = 5/10 = 0.5
### Right now, we will group by postcodes only
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.00,0.00,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000

In [61]:
toronto_grouped.shape

(100, 220)

In [62]:
#11.3 Clustering
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Postcode'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [63]:
#11.4 Briefly check top venues and this will help guide the function to return most common venues based on how many actually are 0 in frequency 

num_top_venues = 5

for post in toronto_grouped['Postcode']:
    print("----"+post+"----")
    
    #Create temp dataframe
    temp = toronto_grouped[toronto_grouped['Postcode'] == post].T.reset_index()
    #Assign coln names
    temp.columns = ['venue','freq']
    
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                      venue  freq
0      Fast Food Restaurant   0.5
1                Print Shop   0.5
2               Music Venue   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----M1C----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2           Accessories Store   0.0
3                 Music Venue   0.0
4    Mediterranean Restaurant   0.0


----M1E----
                 venue  freq
0       Medical Center  0.11
1    Electronics Store  0.11
2       Breakfast Spot  0.11
3         Intersection  0.11
4  Rental Car Location  0.11


----M1G----
               venue  freq
0        Coffee Shop  0.50
1  Convenience Store  0.25
2  Korean Restaurant  0.25
3       Neighborhood  0.00
4      Metro Station  0.00


----M1H----
                  venue  freq
0      Hakka Restaurant  0.12
1    Athletics & Sports  0.12
2  Caribbean Restaurant  0.12
3   Fried Chicken Joint  0.12
4                  Bank  0

In [64]:
#Modified function
#Create function
def return_most_common_venues3(row, num_top_venues):#variables to pass to the function
    row_categories = row.iloc[1:]
    old = row_categories.index[row_categories.values > 0].values[0:num_top_venues] #Convert from Index into an array & return only those Indices that have freq > 0
    
    #return row_categories_sorted.index.values[0:num_top_venues]
    #return data2
    while( len(old) < num_top_venues):
        #print('Length at start of loop:', len(old))
        old = np.append(old, 'NA') #Append NA if no. of most frequent venues < num_top_venues, such that the dataframe can be formed (otherwise uneven number of rows will create problems)
        #print('Length after appending:', len(old))
    return old#Imp. to use return, as then we get back numpy 1d array; if using print, we don't get that...just a regular array I think as shown in cell above
    
#Run briefly
#return_most_common_venues3(row = toronto_grouped.iloc[0, :], num_top_venues = 4)

In [65]:
################################################
#Run the function
num_top_venues = 5
indicators = ['st', 'nd', 'rd']#Suffixes i.e. 1st, 2nd etc

# create columns according to number of top venues
columns = ['Postcode']#['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues3(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(6)

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,NA,NA,NA
1,M1C,Bar,Construction & Landscaping,NA,NA,NA
2,M1E,Breakfast Spot,Electronics Store,Intersection,Medical Center,Mexican Restaurant
3,M1G,Coffee Shop,Convenience Store,Korean Restaurant,NA,NA
4,M1H,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Fried Chicken Joint
5,M1J,Playground,NA,NA,NA,NA


In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#Basically we will attempt to create a dataframe that shows for each postcode, the the most common venues

toronto_new = data_merged #Contains postcode----->borough------>lat----->long

# merge 
toronto_new = toronto_new.join(neighborhoods_venues_sorted.set_index('Postcode'), on='PostalCode')#Because 'PostalCode' was the name of the column in data_merged, & thus toronto_new

toronto_new.tail()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
98,M9N,York,"Weston,,,,,,",43.706876,-79.518188,4.0,Park,NA,NA,NA,NA
99,M9P,Etobicoke,"Westmount,,,,,,",43.696319,-79.532242,0.0,Chinese Restaurant,Coffee Shop,Intersection,Pizza Place,Sandwich Place
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,4.0,Bus Line,Mobile Phone Shop,Park,Pizza Place,NA
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437,0.0,Beer Store,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint,Grocery Store
102,M9W,Etobicoke,"Etobicoke,,,,,,",43.706748,-79.594054,0.0,Bar,Drugstore,Rental Car Location,NA,NA


In [67]:
toronto_new_nonull = toronto_new.dropna(subset= ['Cluster Labels'], axis=0)
toronto_new_nonull['Cluster Labels'].isnull().sum()

0

In [68]:
toronto_new_nonull['Cluster Labels'].dtype

dtype('float64')

#### 12. Visualise k-means clusters

In [69]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_new_nonull['Latitude'], toronto_new_nonull['Longitude'], toronto_new_nonull['PostalCode'], toronto_new_nonull['Cluster Labels'].astype('int')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters